In [1]:
import io
import os
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import rocks
# rocks.set_log_level("error")

import ssptools
import astropy.units as u
from astropy.coordinates import SkyCoord
import fink_utils.sso.spins as finkus

In [2]:
import figure_setup as fs

In [3]:
target = "223"

In [4]:
def dist_reduction(d_obs, d_sun):
    return 5 * np.log10(d_obs * d_sun)

# Get SSOFT 

In [5]:
# Local Configuration
data_fink = "../"
bft_file = os.path.join(data_fink, "data", "ssoBFT-latest.parquet")

In [6]:
# ZTF filters 1: g, 2: r
filters = {"1": "g", "2": "r"}

S_color = {"g": -0.3928, "r": 0.2913}
sun_color = {"g": -0.3044, "r": 0.1903}

color_C = -(sun_color["g"] - sun_color["r"])
color_S = -(S_color["g"] - S_color["r"])

fink_colors = ["#15284F", "#F5622E"]

V_minus_g = -0.2833
V_minus_r = 0.1777

In [7]:
data = pd.read_parquet(os.path.join(data_fink, "data", "ztf", "sso_ZTF.parquet"))

# Get observations and ephemerides

In [8]:
# Get ZTF observations
r = requests.post(
    "https://fink-portal.org/api/v1/sso",
    json={"n_or_d": target, "withEphem": True, "output-format": "json"},
)

# Format output in a DataFrame
ztf = pd.read_json(io.BytesIO(r.content))

In [9]:
# Get the ephemerides
min_JD = ztf.Date.min()
max_JD = ztf.Date.max()
len_JD = max_JD - min_JD
step = 1
nbd = len_JD / step

eph = ssptools.ephemcc(target, ztf.Date, nbd=nbd, step=f"{step}d", observer="I41")

# Lightcurve: mag vs time

In [10]:
# Mag vs Time
fig, ax = plt.subplots(
    3,
    1,
    figsize=fs.figsize(1),
    sharex=True,
    gridspec_kw={
        "top": 0.99,
        "left": 0.075,
        "right": 0.995,
        "bottom": 0.07,
        "hspace": 0.02,
        "height_ratios": [2, 2, 1],
    },
)
# ZTF Colors and Colors
colors = ["#15284F", "#F5622E"]

show_HG = True
show_HG1G2 = True
show_SHG1G2 = True

res_HG = False
res_HG1G2 = True
res_SHG1G2 = True

for fi, filt in enumerate(["g", "r"]):

    # --------------------------------------------------------------------------------
    # Plot ZTF observations
    cond = ztf["i:fid"] == fi + 1
    if filt == "g":
        color_corr = V_minus_g
    else:
        color_corr = V_minus_r

    ax[fi].scatter(
        ztf.loc[cond, "Date"],
        ztf.loc[cond, "i:magpsf"],
        s=20,
        marker="o",
        color=colors[fi],
        label=f"ZTF {filt}",
        rasterized=True
    )

    # --------------------------------------------------------------------------------
    # HG model
    # Get Parameters
    H = data.loc[data.ssnamenr == target, "HG_H_{}".format(filt)].values[0]
    G = data.loc[data.ssnamenr == target, "HG_G_{}".format(filt)].values[0]
    if fi == 0:
        print(f"HG       : {H:5.2f}  {G:4.2f}")

    # Linear Plot
    if show_HG:
        pred_mag = finkus.func_hg(np.radians(eph["Phase"]), H, G) + dist_reduction(
            eph["Dobs"], eph["Dhelio"]
        )
        ax[fi].plot(
            eph["Date"],
            pred_mag,
            color=colors[fi],
            linestyle="dotted",
            label=r"$\texttt{HG}$",
        )

    # Plot residuals
    if res_HG:
        phase = ztf["Phase"]
        pred_mag = finkus.func_hg(np.radians(phase), H, G)
        ax[2].scatter(
            ztf.loc[:, "Date"],
            ztf.loc[:, "i:magpsf"]
            - dist_reduction(ztf.loc[:, "Dobs"], ztf.loc[:, "Dhelio"])
            - pred_mag,
            color=colors[fi],
            alpha=0.1,
            marker="s",
            rasterized=True,
            label=f"HG",
        )

    # --------------------------------------------------------------------------------
    # HG1G2 model
    # Get Parameters
    H = data.loc[data.ssnamenr == target, "HG1G2_H_{}".format(filt)].values[0]
    G1 = data.loc[data.ssnamenr == target, "HG1G2_G1_{}".format(filt)].values[0]
    G2 = data.loc[data.ssnamenr == target, "HG1G2_G2_{}".format(filt)].values[0]
    if fi == 0:
        print(f"HG1G2    : {H:5.2f}  {G1:4.2f}  {G2:4.2f}")

    # Linear Plot
    if show_HG1G2:
        pred_mag = finkus.func_hg1g2(
            np.radians(eph["Phase"]), H, G1, G2
        ) + dist_reduction(eph["Dobs"], eph["Dhelio"])
        ax[fi].plot(
            eph["Date"],
            pred_mag,
            color=colors[fi],
            linestyle="dashed",
            label=r"$\texttt{HG}$$_{\texttt{1}}$$\texttt{G}$$_{\texttt{2}}$",
        )

    # Plot Residuals
    if res_HG1G2:
        pred_mag = finkus.func_hg1g2(np.radians(ztf.loc[cond, "Phase"]), H, G1, G2)
        ax[2].scatter(
            ztf.loc[cond, "Date"],
            ztf.loc[cond, "i:magpsf"]
            - dist_reduction(ztf.loc[cond, "Dobs"], ztf.loc[cond, "Dhelio"])
            - pred_mag,
            color=colors[fi],
            alpha=0.1,
            marker="s",
            rasterized=True,
            label=r"$\texttt{HG}$$_{\texttt{1}}$$\texttt{G}$$_{\texttt{2}}$"
            + f" {filt}",
        )

    # --------------------------------------------------------------------------------
    # SHG1G2 model
    # Get Parameters
    H = data.loc[data.ssnamenr == target, f"SHG1G2_H_{filt}"].values[0]
    G1 = data.loc[data.ssnamenr == target, f"SHG1G2_G1_{filt}"].values[0]
    G2 = data.loc[data.ssnamenr == target, f"SHG1G2_G2_{filt}"].values[0]
    ra0 = data.loc[data.ssnamenr == target, "SHG1G2_alpha0"].values[0]
    dec0 = data.loc[data.ssnamenr == target, "SHG1G2_delta0"].values[0]
    R = data.loc[data.ssnamenr == target, "SHG1G2_R"].values[0]
    rms = data.loc[data.ssnamenr == target, "SHG1G2_rms"].values[0]
    print(
        f"HG21G1  {filt}: {H:5.2f}  {G1:4.2f}  {G2:4.2f}   {ra0:4.1f}  {dec0:4.1f}   {R:4.2f}    {rms:.3f}"
    )

    # Plot
    if show_SHG1G2:
        radec = SkyCoord(eph.RA, eph.DEC, unit=(u.hourangle, u.deg))
        pha = np.transpose(
            [
                [i, j, k]
                for i, j, k in zip(
                    np.radians(eph.Phase),
                    np.radians(radec.ra.deg),
                    np.radians(radec.dec.deg),
                )
            ]
        )
        pred_mag = finkus.func_hg1g2_with_spin(
            pha, H, G1, G2, R, np.radians(ra0), np.radians(dec0)
        ) + dist_reduction(eph["Dobs"], eph["Dhelio"])
        ax[fi].plot(
            eph["Date"],
            pred_mag,
            color=colors[fi],
            label=r"$\texttt{sHG}$$_{\texttt{1}}$$\texttt{G}$$_{\texttt{2}}$",
        )

    # Plot Residuals
    if res_SHG1G2:
        pha = np.transpose(
            [
                [i, j, k]
                for i, j, k in zip(
                    np.radians(ztf.loc[cond, "Phase"]),
                    np.radians(ztf.loc[cond, "RA"]),
                    np.radians(ztf.loc[cond, "Dec"]),
                )
            ]
        )
        pred_mag = finkus.func_hg1g2_with_spin(
            pha, H, G1, G2, R, np.radians(ra0), np.radians(dec0)
        )
        ax[2].scatter(
            ztf.loc[cond, "Date"],
            ztf.loc[cond, "i:magpsf"]
            - dist_reduction(ztf.loc[cond, "Dobs"], ztf.loc[cond, "Dhelio"])
            - pred_mag,
            color=colors[fi],
            rasterized=True,
            label=r"$\texttt{sHG}$$_{\texttt{1}}$$\texttt{G}$$_{\texttt{2}}$"
            + f" {filt}",
        )

    # ax[fi].set_ylabel(f'Apparent magnitude in {filt}')
    ax[fi].legend(ncol=2)  # loc='upper right')


# Residuals
ax[2].axhline(0, color="lightgray", zorder=-100)
ax[2].set_ylim(-0.6, 0.6)
ax[2].legend(ncol=4, loc="upper center")
ax[2].set_ylabel("Residuals")

fig.text(0.01, 0.63, "Apparent magnitude", va="center", rotation="vertical")


# Axes
ax[2].set_xlabel("Time / days")

# --------------------------------------------------------------------------------
# fig.savefig(os.path.join(data_fink, "gfx", "article", f"{target}_vs_time.pgf"))
fig.savefig(
    os.path.join(data_fink, "gfx", "article", f"{target}_vs_time.png"),
    facecolor="white",
    dpi=180,
)
plt.close()

HG       : 10.02  0.00
HG1G2    : 10.18  0.82  0.00
HG21G1  g: 10.65  0.61  0.19   220.8  35.4   0.30    0.068
HG21G1  r: 10.07  0.38  0.30   220.8  35.4   0.30    0.068
